In [1]:
import tensorflow as tf
import keras
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Data/pollen_data.csv').drop(columns='Unnamed: 0')

### Control Pannel

In [3]:
img_size = (300,300)

## Restructure Image Layout

For this project, I plan to use TensorFlow's Image Generator, and the images have to be seperated into a specific folder layout. 

This function Takes all of the images and puts them into the proper format

In [4]:
img_dir = 'Data/images'
def split_to_train_and_val(df,img_dir):
    train, test = train_test_split(df)

    train_pol = train[train['pollen_carrying'] == 1]
    test_pol = test[test['pollen_carrying'] == 1]

    train_no = train[train['pollen_carrying'] == 0]
    test_no = test[test['pollen_carrying'] == 0]


    groups = [
          {'data':train_pol,'dir':'Training/has_pollen'},
         {'data':test_pol,'dir':'Validation/has_pollen'},
          {'data':train_no,'dir':'Training/no_pollen'},
         {'data':test_no,'dir':'Validation/no_pollen'},
    ]


    for group in groups:
        os.system(f"mkdir -p {img_dir}/{group['dir']}")

        for file in group['data']['filename']:
            os.system(f"cp {img_dir}/{file} {img_dir}/{group['dir']}/{file}")

In [5]:
#split_to_train_and_val(df,img_dir)

# Modeling

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        'Data/images',
        target_size=img_size, #img_size is set in the Control Pannel at the top  
        batch_size=128,
        class_mode='binary')


Found 714 images belonging to 2 classes.


In [7]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)))
model.add(tf.keras.layers.MaxPooling2D(2, 2))

model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2)),

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2)),

model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


W0623 19:21:46.346389 140714515551872 deprecation.py:506] From /home/snorks/dev/ml/ds-projects/bees/venv/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

W0623 19:21:46.509164 140714515551872 deprecation.py:323] From /home/snorks/dev/ml/ds-projects/bees/venv/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
